In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models


from models.Generator import Generator
from models.Discriminator import Discriminator

In [31]:
class Content_loss(nn.Module):
    def __init__(self):
        super(Content_loss,self).__init__()
        
        vgg = models.vgg19(pretrained=True)
        vgg.eval()
        self.feature = nn.Sequential(*list(vgg.features.children())[:-1])
        
        self.MSE = nn.MSELoss()
        
    def forward(self, HR, SR):
        HR_feature = self.feature(HR)
        SR_feature = self.feature(SR)
        loss  = self.MSE(HR_feature, SR_feature)
        return loss.sum()
    
class Adversarial_loss(nn.Module):
    def __init__(self):
        super(Adversarial_loss,self).__init__()
        self.D = Discriminator()
        
    def forward(self, SR):
        loss=-torch.log10(D(SR))
        
        return loss.sum()

    
class Perceptual_loss(nn.Module):
    def __init__(self):
        super(Perceptual_loss,self).__init__()
        self.content_loss = Content_loss()
        self.adversarial_loss = Adversarial_loss()
    def forward(self, HR, SR):
        loss = self.content_loss(HR, SR) + 10**-3*self.adversarial_loss(SR)
        return loss
        

## Input

In [32]:
G = Generator()
D = Discriminator()

In [33]:
HR = torch.rand(1,3,256,256)
print(HR.shape)
SR = G(torch.rand(1,32,32,3))
print(SR.shape)

torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])


## loss
### content_loss

In [34]:
c_loss = content_loss()

In [35]:
loss1 = c_loss(HR, SR)
loss1

tensor(0.0145, grad_fn=<MeanBackward0>)

### adversarial_loss

In [36]:
a_loss = adversarial_loss()

In [37]:
loss2 = a_loss(SR)
loss2

tensor(0.2414, grad_fn=<SumBackward0>)

### Perceptual_loss

In [38]:
p = Perceptual_loss()

In [39]:
p(HR, SR)

tensor(0.0147, grad_fn=<AddBackward0>)